# Nighttime Lights Trends

Analyzing conflict dynamics through the lens of [NASA's Black Marble Nighttime Lights](https://blackmarble.gsfc.nasa.gov) dataset opens a unique window into the often-hidden facets of global unrest. In a world marked by diverse forms of conflict, from armed confrontations to civil unrest, the dataset offers an unconventional yet powerful tool for understanding the ripple effects of these conflicts on human settlements and infrastructure. By tracking nighttime light variations and disruptions, we can unearth vital insights into population displacement, economic destabilization, and the societal impacts of conflict. This analysis explores the potential of the Black Marble Nighttime Lights dataset to not only detect areas affected by conflict but also to quantify the extent of its influence on human lives and livelihoods, providing a valuable perspective on the multifaceted consequences of conflict worldwide.

In [1]:
from datetime import datetime
import geopandas
import pandas as pd

from bokeh.models import Div
from bokeh.layouts import gridplot, column as b_column

from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    Legend,
    Range1d,
    Span,
    TabPanel,
    Tabs,
    Text,
    Title,
)
from bokeh.plotting import figure, output_notebook, show
import math
from math import pi

import pandas as pd

from bokeh.models import BasicTicker, PrintfTickFormatter
from bokeh.plotting import figure, show
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import linear_cmap
import colorcet as cc

import numpy as np

In [2]:
group = lambda flat, size: [flat[i:i+size] for i in range(0,len(flat), size)]
COLORS = cc.b_glasbey_hv

## Data

### Black Marble 

[NASA's Black Marble](https://blackmarble.gsfc.nasa.gov) VIIRS (Visible Infrared Imaging Radiometer Suite) Nighttime Lights dataset represents a remarkable advancement in our ability to monitor and understand nocturnal light emissions on a global scale. By utilizing cutting-edge satellite technology and image processing techniques, the Black Marble VIIRS dataset offers a comprehensive and high-resolution view of the Earth's nighttime illumination patterns. 

In [3]:
NTL = pd.read_csv("../../data/nighttimelights/ntl_bm_daily.csv", parse_dates=["date"], index_col=["date"])
NTL = NTL.pivot_table(values=["ntl_mean"], columns=["NAME_2"], index="date")
NTL = NTL.groupby(pd.Grouper(freq="W")).mean()

NTL = NTL[(NTL.index >= "2022-01-01")]

data = 100*(NTL/NTL[(NTL.index >= "2022-01-01") & (NTL.index < "2023-01-01")].mean() - 1)

## Findings



### Percent Change in Average NTL Luminosity 

n this exploratory analysis, we conducted analysis of luminosity trends, comparing the observed luminosity levels to a baseline established in the year 2022.

In [4]:
def create_plot(data):
    p = figure(
        title="Palestine: Percent Change in Nighttime Lights Average Luminosity",
        width=800,
        height=800,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Luminosity Percent change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    p.y_range = Range1d(-100, 100, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi/4
    p.add_layout(
       Title(
           text=f"Percent change (compared to 2022) in NTL luminosity for each second-level administrative division",
           text_font_size="12pt",
           text_font_style="italic",
       ),
       "above",
    )
    p.add_layout(
       Title(
           text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
           text_font_size="10pt",
           text_font_style="italic",
       ),
       "below",
    )
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="red",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips="date: @x{%F}, percent change: @y",
            formatters={"@x": "datetime"},
        )
    )
    renderers = []
    for column, color in zip(data.columns, COLORS):
            r = p.line(
                data.index,
                data[column],
                legend_label=str(column[1]),
                line_color=color,
                line_width=2,
            )
            renderers.append(r)

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "12pt"
    #p.sizing_mode = "scale_both"
    return p

In [5]:
output_notebook()
show(create_plot(data))

Loading BokehJS ...

In [6]:
def create_plot(data, COLORS):
    p = figure(
        title=data.columns[0][1],
        width=800,
        height=800,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Luminosity Percent change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    p.y_range = Range1d(-100, 100, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi/4
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="red",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips="date: @x{%F}, percent change: @y",
            formatters={"@x": "datetime"},
        )
    )
    renderers = []
    for column, color in zip(data.columns, COLORS):
            r = p.line(
                data.index,
                data[column],
                line_color=color,
                line_width=2,
            )
            renderers.append(r)

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "12pt"
    p.sizing_mode = "scale_both"
    return p

In [7]:
plots = list()

for column, color in zip(data.columns, COLORS):
    p = create_plot(data[column].to_frame(), [color])
    plots.append(p)

p = gridplot(group(plots, 4))
p.sizing_mode = 'scale_both'

In [8]:
show(p)

#### Heatmap

In [9]:
df = data.stack().reset_index()
df["date"] = df["date"].astype(str)

In [10]:
# this is the colormap from the original NYTimes plot
#colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
#colors = colors[::-1]

from palettable.colorbrewer.diverging import RdBu_7 as palette
colors = palette.hex_colors

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title=f"Palestine: Weekly Percent Change (compared to 2022 average) in Nighttime Lights Luminosity",
           x_range=df["date"].unique(), y_range=df["NAME_2"].unique(),
           x_axis_location="above", width=1800, height=800,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('date', '@date @NAME_2'), ('rate', '@ntl_mean%')])

p.title.text_font_size = "18pt"
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "16px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

r = p.rect(x="date", y="NAME_2", width=1, height=1, source=df,
           fill_color=linear_cmap("ntl_mean", colors, low=-75, high=75),
           line_color=None)

p.add_layout(
       Title(
           text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
           text_font_size="10pt",
           text_font_style="italic",
       ),
       "below",
    )
p.add_layout(r.construct_color_bar(
    major_label_text_font_size="16px",
    ticker=BasicTicker(desired_num_ticks=len(colors)),
    formatter=PrintfTickFormatter(format="%d%%"),
    label_standoff=6,
    border_line_color=None,
    padding=50,
), 'right')

In [11]:
show(p)

### Point-in-Time Comparison

#### Daily

```{figure} ./figures/pse_ntl_VNP46A1_2023-01-01.png
---
height: 400px
---
Nighttime lights on January 1, 2023. Source: NASA Black Marble (VNP46A1).
```

```{figure} ./figures/pse_ntl_VNP46A1_2023-10-21.png
---
height: 400px
---
Nighttime lights on October 21, 2023. Source: NASA Black Marble (VNP46A1).
```

#### Weekly
We visualize below weekly snapshots of the percent change (compared to 2022) in NTL average luminosity for each second-level administrative division. 

PSE = geopandas.read_file("../../data/boundaries/gadm41_PSE_shp/gadm41_PSE_2.shp")

##### Week of 2023-10-08

In [13]:
PSE.merge(data.iloc[-3].to_frame("luminosity").reset_index(), on="NAME_2").explore(column="luminosity", cmap="seismic_r", vmin=-100, vmax=100)

##### Week of 2023-10-15

In [14]:
PSE.merge(data.iloc[-2].to_frame("luminosity").reset_index(), on="NAME_2").explore(column="luminosity", cmap="seismic_r", vmin=-100, vmax=100)

##### Week of 2023-10-22

In [15]:
PSE.merge(data.iloc[-1].to_frame("luminosity").reset_index(), on="NAME_2").explore(column="luminosity", cmap="seismic_r", vmin=-100, vmax=100)

## Limitations 

Using nighttime lights to estimate macroeconomic indicators during conflict may be a valuable approach, but it comes with several assumptions and limitations. Here's a list of some of the key assumptions and limitations:

```{caution}
**Assumptions:**

- **Luminosity Reflects Economic Activity:** The approach assumes that the level of nighttime lights is a reliable proxy for economic activity. It presupposes that areas with brighter lights correspond to higher economic productivity.

- **Baseline Data Availability:** It assumes the availability of baseline nighttime lights data before the onset of the conflict. The accuracy of the estimates depends on the quality and relevance of this baseline data.

- **Spatial Distribution:** The method assumes that nighttime lights are evenly distributed within a given geographic area and that changes in luminosity accurately reflect changes in economic activity across all locations.
    

**Limitations:**

- **Confounding Factors and Data Interpretation:** The approach may require subjective interpretation, as it may not distinguish between reduced lighting due to conflict and reduced lighting due to other factors. Changes in nighttime lights can be influenced by factors other than economic activity, such as energy conservation measures, urban development, or seasonal variations.

- **Generalization:** The approach might lead to overgeneralization, as a reduction in nighttime lights can be associated with various economic outcomes, from minor disruptions to severe economic downturns.

- **Alternative Explanations:** Changes in nighttime lights can result from factors other than conflict, such as urban development, changes in economic activities, or natural disasters. Therefore, it may not always be clear whether a decline in nighttime lights is solely due to conflict.

- **Geopolitical Factors:** The dataset may be subject to geopolitical biases, with some areas having less comprehensive coverage due to political reasons.

- **Data Lag:** There can be a significant time lag between the occurrence of a conflict event and its reflection in the nighttime lights dataset. This lag may limit the dataset's utility for real-time conflict monitoring.

- **Resolution and Urban Bias:** The dataset's spatial resolution may not be fine enough to capture small villages or isolated conflict events. It may also have an urban bias, making it less suitable for analyzing rural or remote conflicts.
```

To address these assumptions and limitations, it is crucial to complement nighttime lights data analysis with other sources of information and adopt a cautious and context-aware approach when interpreting the findings.

## References

{cite:empty}`ROMAN2018113`

```{bibliography}
:filter: docname in docnames
:style: plain
```